In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from zipfile import ZipFile
file_name='/content/drive/MyDrive/skin_disease_aug.zip'
with ZipFile(file_name, 'r') as zip:
  zip.extractall()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data=ImageDataGenerator(rescale=1/255.,validation_split=0.3)
dir='/content/skin_disease_aug/'

X = Y = 224

training = data.flow_from_directory(dir,
                                    class_mode = "categorical",
                                    target_size = (X, Y),
                                    color_mode="rgb", 
                                    shuffle = True,
                                    subset='training',
                                    seed = 42)

validation = data.flow_from_directory(dir,
                                      class_mode = "categorical",
                                      target_size = (X, Y),
                                      color_mode="rgb", 
                                      subset='validation',
                                      shuffle = True,seed=42)

Found 3188 images belonging to 3 classes.
Found 1364 images belonging to 3 classes.


In [ ]:
training.class_indices

{'Actinic_keratosis': 0, 'Normal': 1, 'skin_cancer': 2}

## **Model1**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
import tensorflow as tf

In [ ]:
classifier=Sequential()
classifier.add(Conv2D(filters=16,kernel_size=3,padding="same",activation="relu",input_shape=(224,224,3)))
classifier.add(MaxPooling2D(pool_size=2))
classifier.add(Conv2D(filters=32,kernel_size=3,padding="same",activation ="relu"))
classifier.add(MaxPooling2D(pool_size=2))
classifier.add(Conv2D(filters=64,kernel_size=3,padding="same",activation ="relu"))
classifier.add(MaxPooling2D(pool_size=2))
classifier.add(Dropout(0.1))
classifier.add(Conv2D(filters=128,kernel_size=3,padding="same",activation="relu"))
classifier.add(MaxPooling2D(pool_size=2))
classifier.add(Dropout(0.3))
classifier.add(Flatten())
classifier.add(Dense(256,activation="relu"))
classifier.add(Dropout(0.3))
classifier.add(Dense(128,activation="relu"))
classifier.add(Dropout(0.3))
classifier.add(Dense(3,activation="softmax"))
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 28, 28, 64)       0

In [ ]:
classifier.load_weights('/content/drive/MyDrive/weights_skin_disease_save.h5')

In [ ]:
classifier.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
train_loss,train_acc=classifier.evaluate(training)

In [ ]:
test_loss,test_acc=classifier.evaluate(validation)

In [ ]:
#classifier.fit(training,validation_data=validation,epochs=5,batch_size=20)

Epoch 1/5
16/16 [==============================] - 4s 257ms/step - loss: 0.2055 - accuracy: 0.9237 - val_loss: 0.5043 - val_accuracy: 0.8443
Epoch 2/5
16/16 [==============================] - 4s 244ms/step - loss: 0.1412 - accuracy: 0.9558 - val_loss: 0.5113 - val_accuracy: 0.8585
Epoch 3/5
16/16 [==============================] - 4s 242ms/step - loss: 0.1296 - accuracy: 0.9618 - val_loss: 0.4848 - val_accuracy: 0.8821
Epoch 4/5
16/16 [==============================] - 4s 242ms/step - loss: 0.1680 - accuracy: 0.9337 - val_loss: 0.6507 - val_accuracy: 0.8585
Epoch 5/5
16/16 [==============================] - 4s 241ms/step - loss: 0.1368 - accuracy: 0.9578 - val_loss: 0.5749 - val_accuracy: 0.8868


In [ ]:
#classifier.save('/content/drive/MyDrive/weights_skin_disease_save.h5')

## **Model2**

In [ ]:
from zipfile import ZipFile
file_name='/content/drive/MyDrive/skin_cancer.zip'
with ZipFile(file_name, 'r') as zip:
  zip.extractall()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data=ImageDataGenerator(rescale=1/255.,validation_split=0.3)

X = Y = 224
dir1 = '/content/skin_cancer/'

training1 = data.flow_from_directory(dir1,
                                    class_mode = "binary",
                                    target_size = (X, Y),
                                    color_mode="rgb", 
                                    shuffle = True,
                                    subset='training',
                                    seed = 42)

validation1 = data.flow_from_directory(dir1,
                                      class_mode = "binary",
                                      target_size = (X, Y),
                                      color_mode="rgb", 
                                      subset='validation',
                                      shuffle = True,seed=42)

Found 2112 images belonging to 2 classes.
Found 905 images belonging to 2 classes.


In [ ]:
training1.class_indices

{'benign': 0, 'malignant': 1}

#### **Transfer learning**

In [ ]:
base_model1=tf.keras.applications.MobileNet(input_shape=(224,224,3),include_top=False)

17235968/17225924 [==============================] - 0s 0us/step


In [ ]:
classifier1=Sequential([base_model1,
                            Flatten(),
                           Dropout(0.3),
                          Dense(512,activation='relu'),
                           Dropout(0.3),
                          Dense(64,activation='relu'),
                           Dropout(0.2),
                          Dense(1,activation='sigmoid')])
classifier1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 50176)             0         
                                                                 
 dropout_8 (Dropout)         (None, 50176)             0         
                                                                 
 dense_9 (Dense)             (None, 512)               25690624  
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 dense_10 (Dense)            (None, 64)                32832     
                                                      

In [ ]:
classifier1.load_weights('/content/drive/MyDrive/weights_skin_cancer.h5')

In [ ]:
classifier1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
train_loss1,train_acc1=classifier1.evaluate(training1)

66/66 [==============================] - 7s 85ms/step - loss: 0.1110 - accuracy: 0.9560


In [ ]:
test_loss1,test_acc1=classifier1.evaluate(validation1)

29/29 [==============================] - 3s 96ms/step - loss: 0.4751 - accuracy: 0.8597


In [ ]:
#transfer_model.fit(training1,validation_data=validation1,epochs=3,batch_size=50)

Epoch 1/3
66/66 [==============================] - 16s 235ms/step - loss: 0.0929 - accuracy: 0.9612 - val_loss: 0.6251 - val_accuracy: 0.8530
Epoch 2/3
66/66 [==============================] - 16s 238ms/step - loss: 0.0864 - accuracy: 0.9744 - val_loss: 0.5845 - val_accuracy: 0.8464
Epoch 3/3
66/66 [==============================] - 16s 238ms/step - loss: 0.0806 - accuracy: 0.9749 - val_loss: 0.4751 - val_accuracy: 0.8597


In [ ]:
#transfer_model.save('/content/drive/MyDrive/weights_skin_cancer.h5')